In [44]:
from data_loader import load_hotel_reserve
import pandas as pd
import numpy as np

In [45]:
customer_tb, hotel_tb, reserve_tb = load_hotel_reserve()
reserve_tb.head()

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price
0,r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200
1,r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600
2,r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600
3,r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400
4,r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100


In [46]:
hotel_tb.head()

,hotel_id,base_price,big_area_name,small_area_name,hotel_latitude,hotel_longitude,is_business
0,h_1,26100,D,D-2,43.064569,141.511397,True
1,h_2,26400,A,A-1,35.715320,139.939446,True
2,h_3,41300,E,E-4,35.281572,136.988565,False
3,h_4,5200,C,C-3,38.431293,140.795615,False
4,h_5,13500,G,G-3,33.597291,130.533872,True


# query()

In [5]:
# combine hotel and reserve table, extract people_num=1 and those hotels are business
df_res = (pd.merge(reserve_tb, hotel_tb, 
                   on='hotel_id', how='inner')
            .query('people_num == 1 & is_business'))

df_res.head()

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,base_price,big_area_name,small_area_name,hotel_latitude,hotel_longitude,is_business
14,r310,h_219,c_73,2017-03-30 01:52:36,2017-04-18,11:00:00,2017-04-21,1,30900,10300,B,B-3,35.644729,139.693389,True
18,r1433,h_219,c_348,2017-04-08 03:19:47,2017-04-25,10:00:00,2017-04-28,1,30900,10300,B,B-3,35.644729,139.693389,True
21,r2992,h_219,c_756,2016-02-17 14:02:23,2016-03-01,12:00:00,2016-03-03,1,20600,10300,B,B-3,35.644729,139.693389,True
24,r3522,h_219,c_875,2016-09-09 07:10:21,2016-09-20,10:00:00,2016-09-21,1,10300,10300,B,B-3,35.644729,139.693389,True
101,r7,h_256,c_1,2017-12-29 10:38:36,2018-01-25,10:30:00,2018-01-28,1,103500,34500,C,C-1,38.237294,140.696131,True


# 条件に応じた結合テーブルの切り替え
タスク：ホテルごとに他のホテルをレコメンドしたい場合の前処理をしましょう

１）対象のデータセットは、ホテルの予約レコードです。

２）ホテルテーブルのすべてのホテルに対して、レコメンド候補のホテルを紐づけたデータを作成します。

３）同じ町のホテルが２０件以上ある場合は、同じ町のホテルをレコメンド候補とします。同じ町のホテルが２０件に満たない場合は、同じ市のホテルをレコメンド候補とします。

In [6]:
small_hotel_mst = (hotel_tb.groupby(['big_area_name','small_area_name'], as_index=False).size()
                           .reset_index()
                           .rename(columns={0:'hotel_cnt'}))
small_hotel_mst['join_area_id'] = np.where(small_hotel_mst['hotel_cnt'] - 1 >= 20, 
                                           small_hotel_mst['small_area_name'], small_hotel_mst['big_area_name'])

small_hotel_mst.head()

,big_area_name,small_area_name,hotel_cnt,join_area_id
0,A,A-1,35,A-1
1,A,A-3,30,A-3
2,B,B-1,15,B
3,B,B-2,18,B
4,B,B-3,19,B


In [7]:
small_hotel_mst = small_hotel_mst.drop(['hotel_cnt', 'big_area_name'], axis=1)
base_hotel_mst = pd.merge(hotel_tb, small_hotel_mst, on='small_area_name')[['hotel_id', 'join_area_id']]

In [48]:
base_hotel_mst.head()

,hotel_id,join_area_id
0,h_1,D
1,h_79,D
2,h_125,D
3,h_127,D
4,h_129,D


In [49]:
recommend_hotel_mst = (pd.concat([hotel_tb[['small_area_name', 'hotel_id']].rename(columns={'small_area_name': 'join_area_id'}),
                                  hotel_tb[['big_area_name', 'hotel_id']].rename(columns={'big_area_name': 'join_area_id'})]))

In [51]:
recommend_hotel_mst = (recommend_hotel_mst.rename(columns={'hotel_id':'rec_hotel_id'})
                                          .sort_values('join_area_id').reset_index(drop=True))

In [61]:
recommend_hotel_mst.sort_values('join_area_id').query('join_area_id == "D"')

,join_area_id,rec_hotel_id
411,D,h_14
401,D,h_265
402,D,h_40
403,D,h_27
410,D,h_275
408,D,h_91
405,D,h_45
406,D,h_22
407,D,h_1
409,D,h_274


In [58]:
base_hotel_mst.sort_values('hotel_id')

,hotel_id,join_area_id
0,h_1,D
79,h_10,A-3
88,h_100,A-3
18,h_101,A-1
251,h_102,B
...,...,...
193,h_95,F
162,h_96,C-1
116,h_97,C-2
17,h_98,A-1


In [40]:
rec_hotel_mst = (pd.merge(base_hotel_mst, recommend_hotel_mst, on='join_area_id')
                   .drop('join_area_id',axis=1).query('hotel_id != rec_hotel_id').reset_index(drop=True))

In [43]:
rec_hotel_mst.head()

,hotel_id,rec_hotel_id
0,h_1,h_109
1,h_1,h_218
2,h_1,h_124
3,h_1,h_129
4,h_1,h_103


In [56]:
pd.merge(base_hotel_mst, recommend_hotel_mst, on='join_area_id')

,hotel_id,join_area_id,rec_hotel_id
0,h_1,D,h_109
1,h_1,D,h_218
2,h_1,D,h_124
3,h_1,D,h_129
4,h_1,D,h_103
...,...,...,...
9659,h_220,F,h_165
9660,h_220,F,h_281
9661,h_220,F,h_163
9662,h_220,F,h_198
